# CS525 FINAL PROJECT
### GROUP MEMBERS: Ted Monyak, Jack Forman




In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import os


# Introduction

### Dataset

### Networks

# Methods



# Results

### Training

### Simple CNN
<style>
img {
    display: block;
    margin-left: 0;
    margin-right: auto;
    width: 30%;
    height: 30%;
}
</style>


Discussion about training accuracy  
![image](Model_SimpleCNN_Best/TestingAccuracy.png)

Discussion about validation accuracy  
![image](Model_SimpleCNN_Best/ValAccuracy.png)

Discussion about testing accuracy  
![image](Model_SimpleCNN_Best/TestingAccuracy.png)

### Deeper CCN
<style>
img {
    display: block;
    margin-left: 0;
    margin-right: auto;
    width: 30%;
    height: 30%;
}
</style>


Discussion about training accuracy  
![image](Model_DeepCNN_Best/TrainingAccuracy.png)

Discussion about validation accuracy  
![image](Model_DeepCNN_Best/ValAccuracy.png)

Discussion about testing accuracy  
![image](Model_DeepCNN_Best/TestingAccuracy.png)


### Transformer
<style>
img {
    display: block;
    margin-left: 0;
    margin-right: auto;
    width: 30%;
    height: 30%;
}
</style>
Discussion about training accuracy ![image](Model_DNATranformer_Best/TrainingAccuracy.png) 
Discussion about validation accuracy ![image](Model_DNATranformer_Best/ValAccuracy.png) 
Discussion about testing accuracy ![image](Model_DNATranformer_Best/TestingAccuracy.png)


### Filters

### Gene differnece in Tissues

# Discussion